# Adding columns to authors.pkl
Some columns need to be added to the authors database. Namely, CN the last mentioned country of the author (university of the last publication), and Title of the most recent publication and the most cited publication.


In [1]:
import pandas as pd
import tqdm
from tqdm.notebook import trange, tqdm
from tqdm import tqdm
import re

In [2]:
df_auth = pd.read_pickle("../data/dataframes/authors_full.pkl")


In [ ]:
df_full = pd.read_pickle("../data/dataframes/full_pubs.pkl")


In [ ]:
df_full

In [12]:

reg = re.compile(r"(\[([^\[\]]+)])? (?P<adress>[^\[\];]+);?")
dic_country = {'Peoples R China': "China",
               'England': 'United Kingdom',
               'Scotland': 'United Kingdom',
               'Wales': 'United Kingdom',
               'Northen Ireland': 'United Kingdom',
               'Northern Ireland': 'United Kingdom',
               'North Ireland': 'United Kingdom',
               'U Arab Emirates': 'United Arab Emirates',
               'Bosnia & Herceg': 'Bosnia and Herzegovina',
               'Trinidad Tobago': 'Trinidad and Tobago',
               'North Macedonia': 'Macedonia',
               'Papua N Guinea': 'Papua New Guinea',
               'DEM REP CONGO': 'Congo [DRC]',
               'Rep Congo': 'Congo [DRC]',
               'BELARUS': 'Belarus',
               'Cote Ivoire': 'Cote d\'Ivoire',
               'Marshall Island': 'Marshall Islands',
               'Dominican Rep': 'Dominican Republic',
               'Turks & Caicos': 'Turks and Caicos Islands',
               'St Helena': 'Saint Helena',
               'St Kitts & Nevi': 'Saint Kitts and Nevis',
               'St Vincent': 'Saint Vincent and the Grenadines',
               'Antigua & Barbu': 'Antigua and Barbuda',
               'Cent Afr Republ': 'Central African Republic',
               'Neth Antilles': 'Netherlands Antilles',
                   }

In [13]:
# Iterating over auth rows, then linking to relevant publication to find country, most cited and most recent title of pub
# dic = {}
# for row in tqdm(df_auth.itertuples(name=None), total=df_auth.shape[0]):
def create_dic_additional_info(name, lst_pubs):
    # email = row[0]
    # set_names = row[1]
    # set_pubs = row[3]
    set_names = name
    set_pubs= lst_pubs
    most_recent_pub_title, most_cited_pub_title = "", ""
    most_recent_pub_year, most_cited_pub_tot_citation, most_cited_pub_year = 0, -1, 0
    # Dic {country:year}
    dic_country_year = {}
    # Iterating over publications
    for pub_id in set_pubs:
        pub = df_full.query('UT == @pub_id')

        # Variables of publication year, title, and total citation
        py, ti, tc = pub.PY.values[0],pub.TI.values[0], pub.TC.values[0]


        # Updating variable when iterating over pubs
        if py >= most_recent_pub_year:
            most_recent_pub_title = ti
            most_recent_pub_year = py
        if tc > most_cited_pub_tot_citation:
            most_cited_pub_tot_citation = tc
            most_cited_pub_title = ti
            most_cited_pub_year = py

        # About countries
        c1_col = pub.C1.values[0]
        # Iterating over addresses in c1_col
        for addr in re.finditer(reg, c1_col):
            for name in set_names:
                if name in addr.group(0):
                    c = addr.group().split(', ')[-1].replace(";", "")
                    if "USA" in c:
                        c = "United States"
                    elif c in dic_country:
                        c= dic_country[c]
                    dic_country_year[c] = py
    if len(dic_country_year) > 0:
        country = max(dic_country_year, key=dic_country_year.get)
    else:
        country = "unknown"
    # dic = dict()
    # dic[email] = {'CN':country, 'MCTI':most_cited_pub_title, 'MCPY':most_cited_pub_year, 'MRTI': most_recent_pub_title, 'MRPY': most_recent_pub_year}

    return country, most_cited_pub_title, most_cited_pub_year, most_recent_pub_title, most_recent_pub_year





In [14]:
def add_country(set_names, set_pubs):
    dic_country_year = {}

    # Iterating over publications
    for pub_id in set_pubs:
        pub = df_full.query('UT == @pub_id')
        py = pub.PY.values[0]
        # About countries
        c1_col = pub.C1.values[0]
        # Iterating over addresses in c1_col
        for addr in re.finditer(reg, c1_col):
            for name in set_names:
                if name in addr.group(0):
                    c = addr.group().split(', ')[-1].replace(";", "")
                    if "USA" in c:
                        c = "United States"
                    elif c in dic_country:
                        c = dic_country[c]
                    dic_country_year[c] = py
    if len(dic_country_year) > 0:
        country = max(dic_country_year, key=dic_country_year.get)
    else:
        country = "unknown"
    return country

In [13]:
%%time
tqdm.pandas()
df_auth['CN'],df_auth['MCTI'],df_auth['MCPY'],df_auth['MRTI'], df_auth['MRPY'] = zip(*df_auth.progress_apply(lambda x: create_dic_additional_info(x['name'], x['lst_publications']), axis=1))

100%|██████████| 1000/1000 [01:17<00:00, 12.84it/s]

CPU times: user 1min 17s, sys: 91.5 ms, total: 1min 17s
Wall time: 1min 17s


In [7]:
import numpy as np

lst = np.array_split(df_auth, 30)

In [8]:
lst[0]

,name,total_publications,lst_publications,PY<2019,PY>=2019,total_citations,keywords,IOT,AI,computing_infrastructure,...,SDG14,SDG15,SDG16,SDG17,Society,Economy,Environment,SDG,DT,Intersection
barbara.klik@uwm.edu.pl,"{Klik, Barbara K., Klik, Barbara}",2,"{WOS:000458909900006, WOS:000570328900001}",0,2,20,"{'Environmental Sciences': 1, 'Public, Environ...",0,0,0,...,0,0,0,0,1,1,0,1,0,0
dorotak@uwm.edu.pl,"{Kulikowska, D., Kulikowska, Dorota}",7,"{WOS:000458909900006, WOS:000689384300001, WOS...",3,4,47,"{'Environmental Sciences': 4, 'Public, Environ...",0,0,0,...,0,0,0,0,5,3,0,1,0,0
mariusz.gusiatin@uwm.edu.pl,"{Gusiatin, Zygmunt M.}",5,"{WOS:000458909900006, WOS:000459141600006, WOS...",1,4,41,"{'Environmental Sciences': 3, 'Public, Environ...",0,0,0,...,0,0,0,0,3,1,1,1,0,0
tvitko@cox.net,"{Vitko, Tadeo G.}",1,{WOS:000635505300001},0,1,2,"{'Engineering, Environmental': 1, 'Environment...",0,0,0,...,0,0,0,0,0,1,0,1,0,0
eandreou3@gmail.com,"{Andreou, E.}",1,{WOS:000410925400094},1,0,6,"{'Construction & Building Technology': 1, 'Eng...",0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
joao.gomes.ferreira@tecnico.ulisboa.pt,"{Ferreira, Joao Gomes}",1,{WOS:000676900300001},0,1,0,{'Green & Sustainable Science & Technology': 1...,0,0,0,...,0,0,0,0,0,1,0,1,0,0
mario.matos@tecnico.ulisboa.pt,"{de Matos, Mario}",2,"{WOS:000412935300016, WOS:000676900300001}",1,1,5,{'Green & Sustainable Science & Technology': 2...,0,0,0,...,0,0,0,0,1,1,0,1,0,0
unaiob@gmail.com,"{Ortega, Unai}",1,{WOS:000676359600001},0,1,0,{'Forestry': 1},0,0,0,...,0,1,0,0,0,1,1,1,0,0
isabel.salcedo@ehu.eus,"{Salcedo, Isabel}",1,{WOS:000676359600001},0,1,0,{'Forestry': 1},0,0,0,...,0,1,0,0,0,1,1,1,0,0


In [5]:
test = df_auth[['name', 'lst_publications']].to_dict(orient='split')

# test['data'] list (list (set(names), set(pubs) ) )
for names, pubs in test['data']:
    print(names)
    print(pubs)
    break

{'Klik, Barbara K.', 'Klik, Barbara'}
{'WOS:000570328900001', 'WOS:000458909900006'}


In [13]:
import multiprocessing
from multiprocessing import Pool

In [14]:
n_cores = multiprocessing.cpu_count()
n_cores

16

In [ ]:
%%time

def multi_run_wrapper(args):
   return add_country(*args)

pool = Pool(16)
results = pool.map(multi_run_wrapper, test['data'])

In [5]:
%%time
with multiprocessing.Pool(processes=8) as pool:
        results = pool.starmap(add_country, test['data'])


NameError: name 'add_country' is not defined

In [19]:
def add_country(set_names, set_pubs):
    dic_country_year = {}

    # Iterating over publications
    for pub_id in set_pubs:
        py, c1 = dic_full[pub_id].values()
        # Iterating over addresses in c1_col
        for addr in re.finditer(reg, c1):
            for name in set_names:
                if name in addr.group(0):
                    c = addr.group().split(', ')[-1].replace(";", "")
                    if "USA" in c:
                        c = "United States"
                    elif c in dic_country:
                        c = dic_country[c]
                    dic_country_year[c] = py
    if len(dic_country_year) > 0:
        country = max(dic_country_year, key=dic_country_year.get)
    else:
        country = "unknown"
    return country

In [ ]:
# df_full to dict
dic_full = df_full.set_index('UT')[['PY', 'C1']].to_dict(orient='index')

In [8]:
# df_auth to dict
auth = df_auth.sample(5)
tuple_auths = auth[['name', 'lst_publications']].to_dict(orient='split')

In [7]:
tuple_auths

{'index': ['mayanfang@hebut.edu.cn',
  'chirstelle.bloch@femto-st.fr',
  'aaksu@istanbul.edu.tr',
  'vasja.rant@ef.uni-lj.si',
  'dbrohlin@sfedu.ru'],
 'columns': ['name', 'lst_publications'],
 'data': [[{'Ma, Yanfang'}, {'WOS:000469901200088', 'WOS:000543421400108'}],
  [{'Bloch, Christelle'}, {'WOS:000398618300043'}],
  [{'Aksu, Abdullah'}, {'WOS:000412880600017'}],
  [{'Rant, Vasja'}, {'WOS:000535273800009'}],
  [{'Rokhlin, Dmitry B.'}, {'WOS:000614019800001'}]]}

In [10]:
import numpy as np
lst = ['France', 'Germany', 'United Kingdom', 'United States', 'Poland']
auth['CN'] = np.array(lst)
auth

,name,total_publications,lst_publications,PY<2019,PY>=2019,total_citations,keywords,IOT,AI,computing_infrastructure,...,SDG15,SDG16,SDG17,Society,Economy,Environment,SDG,DT,Intersection,CN
m.stenzel@unsw.edu.au,"{Stenzel, Martina H.}",3,"{WOS:000592313600012, WOS:000674974800001, WOS...",0,3,30,"{'Materials Science, Biomaterials': 1, 'Materi...",0,0,0,...,0,0,0,0,0,0,0,1,0,France
vnshukla@eci.gov.in,"{Shukla, Seema}",1,{WOS:000380430600021},1,0,0,"{'Computer Science, Artificial Intelligence': ...",1,1,0,...,0,0,0,0,0,0,0,1,0,Germany
ghjahng@gmail.com,"{Jahng, Geon-Ho}",1,{WOS:000561623900002},0,1,2,"{'Clinical Neurology': 1, 'Neurosciences': 1}",0,1,0,...,0,0,0,0,0,0,0,1,0,United Kingdom
jgriffin5@luc.edu,"{Griffin, Jennifer J.}",2,"{WOS:000484407900001, WOS:000495452500001}",0,2,17,"{'Business': 2, 'Environmental Studies': 1, 'M...",0,0,0,...,0,0,0,1,0,1,1,0,0,United States
heloise.legoff@mffp.gouv.qc.ca,"{Le Goff, Heloise}",1,{WOS:000365035900019},1,0,10,{'Forestry': 1},0,0,0,...,0,0,0,0,0,1,1,0,0,Poland


In [17]:
def multi_run_wrapper(args):
    return add_country(*args)

In [18]:
import time
import re

start = time.time()
pool = Pool(16)
results = pool.map(multi_run_wrapper, tuple_auths)
end = time.time()
print(end - start)
print(results)

ItalyItalyAustraliaSouth KoreaJapanChinaUnited StatesIndiaIndiaIndia



1.3306992053985596
['Japan', 'Italy', 'Australia', 'Italy', 'South Korea', 'United States', 'India', 'China', 'India', 'India']








In [1]:
import pandas as pd
input_path = "../data/dataframes/authors/authors_w_cn/0.pkl"
df = pd.read_pickle(input_path).sample(10)

In [10]:
df_full = pd.read_pickle("../data/dataframes/full_pubs.pkl")
# df_full to dict

dic_full = df_full.set_index('UT')[['PY', 'TC', 'TI']].to_dict(orient='index')

In [12]:
auth = df[['lst_publications']].to_dict(orient='split')
tuple_auths = auth['data']

In [18]:
def add_most_cited_pub(set_pubs):

    most_cited_pub_title = ""
    most_cited_pub_tot_citation, most_cited_pub_year = -1, -1
    for pub_id in set_pubs:
        py, tc, ti = dic_full[pub_id].values()
        if tc > most_cited_pub_tot_citation:
            most_cited_pub_tot_citation = tc
            most_cited_pub_title = ti
            most_cited_pub_year = py
    return most_cited_pub_title, most_cited_pub_year, most_cited_pub_tot_citation

In [20]:
ti, py, tc = zip(*[add_most_cited_pub(lst) for lst in tuple_auths])
ti

('Impact of bioethanol impurities on steam reforming for hydrogen production: A review',
 'Sustainable operations of industrial symbiosis: an enterprise input-output model integrated by agent-based simulation',
 'Optimization of two-phased anaerobic sludge digestion using the pressurized ultra filtration membrane with a mesh screen (MS-PUFM)',
 'On the friction drag reduction mechanism of streamwise wall fluctuations',
 'Sustainable Brand Management of Alimentary Goods',
 'Comparison of environmental benchmarks of masonry and concrete structure based on a building model',
 'Microclimate tourism and microclimate tourism security and safety in China',
 'Heterogeneous effects of energy efficiency and renewable energy on carbon emissions: Evidence from developing countries',
 'Structural Behavior of Reinforced Concrete Beams Incorporating Foamed Glass as Aggregates',
 'Essential new PLS-SEM analysis methods for your entrepreneurship analytical toolbox')

In [21]:
from multiprocessing import Pool
with Pool(8) as pool:
    results = pool.map(add_most_cited_pub, tuple_auths)


In [30]:
import numpy as np
ti, py, tc = zip(*results)
df['TI'], df['PY'], df['TC'] = np.array(ti), np.array(py), np.array(tc)
df

,name,total_publications,lst_publications,PY<2019,PY>=2019,total_citations,keywords,IOT,AI,computing_infrastructure,...,Society,Economy,Environment,SDG,DT,Intersection,CN,TI,PY,TC
martha.cobo@unisabana.edu.co,"{Cobo, Martha}",7,"{WOS:000405369800027, WOS:000524161700005, WOS...",3,4,86,"{'Chemistry, Physical': 2, 'Electrochemistry':...",0,0,0,...,0,7,2,1,0,0,Colombia,Impact of bioethanol impurities on steam refor...,2020,34
d.m.yazan@utwente.nl,"{Yazan, Devrim Murat}",10,"{WOS:000434299000006, WOS:000390628300008, WOS...",6,4,267,"{'Environmental Sciences': 4, 'Engineering, En...",0,0,0,...,0,10,5,1,0,0,Netherlands,Sustainable operations of industrial symbiosis...,2020,42
geebhan@catholic.ac.kr,"{Han, Gee-Bong}",7,"{WOS:000284097000002, WOS:000401991400008, WOS...",7,0,24,"{'Engineering, Chemical': 2, 'Engineering, Env...",0,0,0,...,7,1,0,1,0,0,South Korea,Optimization of two-phased anaerobic sludge di...,2016,10
tamas.jozsa@eng.ox.ac.uk,"{Jozsa, Tamas Istvan}",1,{WOS:000629999200013},0,1,1,"{'Thermodynamics': 1, 'Engineering, Mechanical...",0,0,0,...,0,1,0,1,0,0,United Kingdom,On the friction drag reduction mechanism of st...,2020,1
jana.majerova@fpedas.uniza.sk,"{Majerova, Jana}",3,"{WOS:000763773400071, WOS:000516824600114, WOS...",1,2,10,{'Green & Sustainable Science & Technology': 1...,1,0,0,...,1,1,1,1,1,1,Slovakia,Sustainable Brand Management of Alimentary Goods,2020,9
sazedj@fa.ulisboa.pt,"{Sazedj, Soheyl}",1,{WOS:000400033700004},1,0,10,"{'Construction & Building Technology': 1, 'Eng...",0,0,0,...,1,2,0,1,0,0,Portugal,Comparison of environmental benchmarks of maso...,2017,10
1601108007@hqu.edu.cn,{no_match},1,{WOS:000469890300009},0,1,9,"{'Environmental Studies': 1, 'Hospitality, Lei...",0,0,0,...,0,2,0,1,0,0,unknown,Microclimate tourism and microclimate tourism ...,2019,9
fahad.rabiyaphd@yahoo.com,"{Akram, Rabia}",4,"{WOS:000505696700042, WOS:000657366000004, WOS...",0,4,416,"{'Construction & Building Technology': 1, 'Gre...",0,0,0,...,2,3,5,1,0,0,China,Heterogeneous effects of energy efficiency and...,2020,124
dawood.sulaiman@uod.ac,"{Atrushi, Dawood S.}",1,{WOS:000625327600006},0,1,0,{'Multidisciplinary Sciences': 1},0,0,0,...,0,1,0,1,0,0,Iraq,Structural Behavior of Reinforced Concrete Bea...,2021,0
wmcdowell@bradley.edu,"{McDowell, William C., Williams, Ralph I., Jr.}",4,"{WOS:000546201700001, WOS:000590683000041, WOS...",0,4,63,{'Green & Sustainable Science & Technology': 1...,0,0,0,...,0,4,0,1,0,0,United States,Essential new PLS-SEM analysis methods for you...,2021,46


In [26]:
path = "../data/dataframes/authors_w_cn_ti.csv"
df = pd.read_csv(path, sep='\t', index_col=0)
df

,name,total_publications,lst_publications,PY<2019,PY>=2019,total_citations,IOT,AI,computing_infrastructure,additive_manufacturing,...,Society,Economy,Environment,SDG,DT,Intersection,CN,TI,PY,TC
muflih.adnan@uii.ac.id,"Adnan, A.",1,WOS:000701992100008,0,1,0,0,0,0,0,...,0,1,0,1,0,0,Indonesia,Transformation of low-rank coal to clean synga...,2021,0
oki.muraza@pertamina.com,"Muraza, Oki",1,WOS:000701992100008,0,1,0,0,0,0,0,...,0,1,0,1,0,0,Indonesia,Transformation of low-rank coal to clean synga...,2021,0
hushaowei@cqu.edu.cn,"Hu, Shaowei",2,"WOS:000542949400047,WOS:000579071300058",0,2,204,0,1,0,0,...,0,1,0,1,1,1,China,Properties of fresh and hardened fly ash/slag ...,2020,122
zhangpeng@zzu.edu.cn,"Zhang, Peng",5,"WOS:000703965300002,WOS:000515503100028,WOS:00...",0,5,247,0,4,0,0,...,0,1,0,1,1,1,China,Properties of fresh and hardened fly ash/slag ...,2020,122
guojinjun@zzu.edu.cn,"Guo, Jinjun",3,"WOS:000555915200128,WOS:000542949400047,WOS:00...",0,3,206,0,1,0,0,...,1,1,0,1,1,1,China,Properties of fresh and hardened fly ash/slag ...,2020,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ongsoonan@yahoo.com,"Oon, Y. -S.,Ong, Soon-An,Oon, Yoong-Ling,Ong, ...",15,"WOS:000209031000002,WOS:000388591300013,WOS:00...",14,1,426,0,0,0,0,...,15,1,0,1,0,0,Malaysia,Hybrid system up-flow constructed wetland inte...,2015,122
g.feng@hud.ac.uk,"Feng, Guojin",1,WOS:000701700500004,0,1,29,0,0,0,0,...,0,1,0,1,0,0,United Kingdom,A review on online state of charge and state o...,2021,29
jose.a.oliveira@unesp.br,"de Oliveira, Jose Augusto",1,WOS:000474680100032,0,1,22,0,0,0,0,...,0,1,0,1,0,0,Brazil,"Cleaner Production practices, motivators and p...",2019,22
dash.prasanjita@mahindraelectric.com,"Dash, Prasanjita",1,WOS:000724523200057,0,1,0,0,0,0,0,...,0,1,0,1,0,0,India,The way forward for e-mobility in India,2019,0


2687881